- dataset: https://korquad.github.io/category/1.0_KOR.html
- elasticsearch: https://www.elastic.co/docs/deploy-manage/deploy/self-managed/install-elasticsearch-docker-basic

In [ ]:
# docker run -d --name es01 \
#   --net elastic \
#   -p 9200:9200 -m 1GB \
#   -e "discovery.type=single-node" \
#   -e "xpack.security.enabled=false" \
#   -e "xpack.security.http.ssl.enabled=false" \
#   docker.elastic.co/elasticsearch/elasticsearch:9.0.0


In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

from elasticsearch import Elasticsearch
from datasets import load_dataset

In [2]:
ds = load_dataset('json', data_files='..\\data\\korquad\\KorQuAD_v1.0_train.json')

In [3]:
documents = []
korquad_data = ds['train'][0]['data']  # ✔️ 이게 실제 문서 리스트

for item in korquad_data:
    for para in item['paragraphs']:
        context = para['context']
        documents.append(Document(page_content=context))

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", ".", " "]
)

chunks = text_splitter.split_documents(documents)

In [5]:
embeddings = HuggingFaceEmbeddings(
    model_name="../ai_models/base_models/BGE-m3-ko",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
es = Elasticsearch("http://127.0.0.1:9200")


In [6]:
es.options(ignore_status=400).indices.create(
    index="korquad",
    mappings={
        "properties": {
            "content": {"type": "text"},
            "embedding": {
                "type": "dense_vector",
                "dims": 1024,  # 모델에 따라 조정
                "index": True,
                "similarity": "cosine"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'korquad'})

In [9]:
from elasticsearch import helpers
from tqdm import tqdm

actions = []
for chunk in tqdm(chunks):
    text = chunk.page_content
    vector = embeddings.embed_documents([text])[0]  # ✅ 문서용

    actions.append({
        "_index": "korquad",
        "_source": {
            "content": text,
            "embedding": vector
        }
    })

helpers.bulk(es, actions)


100%|██████████| 13981/13981 [2:49:04<00:00,  1.38it/s] 


(13981, [])

In [14]:
query = "gdpr 대해 알려줘"
query_vector = embeddings.embed_query(query)

response = es.search(
    index="korquad",
    knn={
        "field": "embedding",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 10
    }
)

# 결과 출력
for hit in response["hits"]["hits"]:
    print(f"점수: {hit['_score']:.4f}")
    print(f"문서 내용: {hit['_source']['content']}\n")


점수: 0.6783
문서 내용: GPU는 펌웨어 이미지를 통해 접근이 가능하며, 이 이미지는 SD 카드로부터 부팅할때 GPU에 로드된다. 이 펌웨어 이미지는 바이너리 블롭으로도 알려져 있는데, 리눅스용 드라이버는 공개되지 않은 사유 소프트웨어이다. 응용 소프트웨어를 사용하게 되면, 비공개 실시간 라이브러리를 호출하게되고, 이는 다시 리눅스 내의 오픈 소스 드라이버를 호출하게 된다. 제공되는 커널 드라이버의 API가 이런 비공개 라이브러리를 지원하기 위해 특화되어 있다. 비디오 응용 프로그램은 OpenMAX를 사용하며, 3D 그래픽은 OpenGL ES를 사용하고, 2D 응용 프로그램은 OpenVG를 사용한다. OpenVG는 다시 EGL을 사용하게 된다. OpenMAX와 EGL은 다시 커널의 오픈소스 커널 드라이버를 사용하게된다.

점수: 0.6781
문서 내용: 이에 대해 김경수는 사실이 알려진 2018년 4월 14일 오후 9시 30분 국회 정론관에서 기자회견을 하면서 자신에 관한 보도를 해명했다. 김경수는 총선이 열린 2016년부터 인터넷 댓글 조작범 김동원(필명: 드루킹)을 만나 범인이 운영하는 파주의 느릅나무 출판사 사무실을 방문하였다고 주장했다. 김경수는 댓글조작 사건의 범인 김동원(드루킹)에게서 일본 오사카 총영사를 청탁받아 청와대에 추천한 것은 사실이지만, 청와대에서 어렵다는 연락을 받아 이를 전해줬다고 해명했다. 청와대는 "댓글을 조작한 김동원(드루킹)이 주(駐)오사카 총영사로 김경수 의원에게 추천한 인사를 2018년 2월에 청와대 연풍문 2층에서 1시간가량 직접 만났으나 적합하지 않다고 판단했다."고 해명했다.

점수: 0.6745
문서 내용: 박동열 대전지방국세청장에게 정보담당 경찰관, 개인사업자 등 6명이 '증권가 지라시'와 풍문을 전달하였고, 조응천 대통령비서실 공직기강비서관은 박관천 공직기강비서관실 행정관에게 동향파악을 지시하였다. 이에 박관천은 박동열에게 관련 사실을 확인하였고, 박관천은 박동열로부터 전달받은 풍문과 정보를 과장하고 추가하